In [50]:
import tokenize
import os
import pandas as pd
from IPython.display import clear_output
from collections import Counter

def get_token_list(filePath):
    with open(filePath, 'rb') as f:
        tokens = tokenize.tokenize(f.readline)
        file_tokens = []
        for token in tokens:
            file_tokens.append(tokenize.tok_name[token.type])
            
    return file_tokens

In [10]:
file_path = "C:/Users/mdn769/BugFixStatistics/SCGrunable/runable/"

In [35]:
''' This will find a list of unique token sequences (TS Features) from all the files in this subject system. Each item of the list will be
the feature name of the dataset to be prepared. 
'''
list_ts_features = []
count = 0
inPath = file_path+ "patch_source/openstack/added/"
totalFiles = len(os.listdir(inPath))
for file in os.listdir(inPath):   
#     if(count == 5):
#         break
    #print(file.split('.')[-1])
    if(file.split('.')[-1]=="py"):
        clear_output(wait=True)
        print("Working for: ("+str(count)+"/"+str(totalFiles)+")" + ", "+ file)    
        try:
            file_tokens = get_token_list(inPath+file)
#             print(file_tokens)
#             print("\n\n******\n\n")
                        
            for ngram in [2, 3]: #n-gram = 2 & 3
                temp=zip(*[file_tokens[i:] for i in range(0, ngram)]) 
                ans=[' '.join(ngram) for ngram in temp]
            
            for a in ans:
                if(a not in list_ts_features):
                    list_ts_features.append(a)
                    
        except: print() #do nothing...

    count+=1
    


Working for: (3121/3122), swift_fa3c871f0b1544f859bacf38497580afd69ced0b.py



In [36]:
print(len(list_ts_features), list_ts_features)

213 ['ENCODING NAME NAME', 'NAME NAME OP', 'NAME OP NAME', 'OP NAME NAME', 'NAME NAME NAME', 'NAME OP NEWLINE', 'OP NEWLINE NAME', 'NEWLINE NAME OP', 'OP NAME OP', 'NAME OP OP', 'OP OP NEWLINE', 'NAME OP NL', 'OP NL STRING', 'NL STRING OP', 'STRING OP NL', 'OP NL NAME', 'NL NAME OP', 'NAME OP STRING', 'OP STRING NEWLINE', 'STRING NEWLINE NAME', 'NEWLINE NAME NAME', 'OP OP NAME', 'OP STRING OP', 'STRING OP OP', 'OP NAME NEWLINE', 'NAME NEWLINE NAME', 'OP STRING NL', 'STRING NL STRING', 'STRING OP NEWLINE', 'NEWLINE NAME NEWLINE', 'OP OP NUMBER', 'OP NUMBER OP', 'NUMBER OP NEWLINE', 'NAME OP NUMBER', 'NAME NAME NEWLINE', 'OP NEWLINE STRING', 'NEWLINE STRING NEWLINE', 'OP NL OP', 'NL OP STRING', 'STRING OP STRING', 'STRING OP NAME', 'NUMBER OP NUMBER', 'NUMBER OP OP', 'OP OP NL', 'STRING OP NUMBER', 'NUMBER OP NL', 'OP OP OP', 'OP NEWLINE ENDMARKER', 'OP OP STRING', 'OP NEWLINE OP', 'NEWLINE OP NAME', 'STRING NEWLINE OP', 'OP NAME NL', 'NAME NL OP', 'NL OP NEWLINE', 'OP NEWLINE COMMENT', 

In [57]:
''' Prepare the dataset. For each commit, list_pattern is the list of feature values. 
'''
dataset = [["commit_id", "submodule"] + list_ts_features] # prepare the top row of the dataset. 
#subject_systems=["bitcoin", "jenkins", "litecoin", "lucene", "mongo", "oozie"]
subject_systems=["openstack"]
#commit_type=["added", "deleted"]
commit_type=["added"]
not_worked=""
# fileContent = open("ssNames.txt", "r", encoding='utf-8').read()
# #print("Length of File Content", len(fileContent))
# ssNames = fileContent.split('\n')

for ss in subject_systems:
    if(len(ss.strip()) == 0): #Ignore the last empty line....
        break
    for ctype in commit_type:
        inPath = file_path+ "patch_source/"+ss+"/"+ctype+"/"
        #outPath = "xml_gml/"+ss+"/"+ctype+"/"
        
        count = 0
        totalFiles = len(os.listdir(inPath))
        for file in os.listdir(inPath):   
#             if(count == 10):
#                 break
            #print(inPath+file)
            if(file.split('.')[-1]=="py"):
                clear_output(wait=True)
                print("Working for: ("+str(count)+"/"+str(totalFiles)+")"+ss+", "+ctype+", "+file)    
                try:
                    file_tokens = get_token_list(inPath+file)

                    list_ts = []
                    for ngram in [2, 3]: #n-gram = 2 & 3
                        temp=zip(*[file_tokens[i:] for i in range(0, ngram)]) 
                        list_ts +=[' '.join(ngram) for ngram in temp]
                    
                    countedValues = Counter(list_ts)
                    #print(countedValues)
                    
                    #file := 'submodule_commitid_ext: swift_fa3c871f0b1544f859bacf38497580afd69ced0b.py'
                    result_row = [file.split('.')[0].split('_')[1], file.split('.')[0].split('_')[0]] #Take only the commit name from the file
                    ZeroLine = 0 #Checks if all the values in this row is zero
                    for lp in list_ts_features:
                        if(countedValues[lp] > 0):
                            ZeroLine += 1
                            result_row.append(countedValues[lp]) 
                        else: 
                            result_row.append(0)
                    
                    if(ZeroLine > 0):
                        dataset.append(result_row)

                except: print() #do nothing...
                    
            count+=1
                
#print(dataset)

Working for: (3121/3122)openstack, added, swift_fa3c871f0b1544f859bacf38497580afd69ced0b.py



In [53]:
print(ss)
print(not_worked)

openstack



In [58]:
pd.DataFrame(dataset).to_csv(ss+'_ts_dataset.csv', index=False, header=False)